# Sift Client Basic Example

This notebook demonstrates the core features of the Sift Python client:
- Initializing the Sift client
- Finding assets
- Finding runs
- Searching channels
- Pulling data
- Creating calculated channels
- Setting rules

## Running this notebook

This notebook is written in Jupyter Notebook format and can be run in any Jupyter environment.

Some additional package prerequisites are required to run this notebook:
- `notebook` for running Jupyter Notebooks
- `python-dotenv` for loading environment variables
- `rich` for pretty-printing output
- `pandas` for data manipulation and analysis
- `matplotlib` for data visualization

You can install these packages using `pip install notebook rich python-dotenv pandas matplotlib`.

## Setup and Initialization

First, import the necessary modules and initialize the Sift client with your credentials.

Best practice is to access credentials using environment variables or a `.env` file with `python-dotenv`. Avoid hardcoding your API in any code you write.

In [1]:
import os
from datetime import datetime, timedelta
from rich import print
from dotenv import load_dotenv
from sift_client import SiftClient
from sift_client.sift_types import (
    ChannelReference,
    CalculatedChannelCreate,
    RuleCreate,
    RuleAction,
    RuleAnnotationType,
)

/Users/alexluck/Projects/sift-clients/python/lib/sift_client/resources/_base.py:9: SiftExperimentalWarning: `sift_client` is experimental and is subject to change. Use with caution.
  _sift_client_experimental_warning()


In [2]:
# Get our environment variables
load_dotenv()  # Load environment variables from .env file
api_key = os.getenv("SIFT_API_KEY")
grpc_url = os.getenv("SIFT_GRPC_URI")
rest_url = os.getenv("SIFT_REST_URI")

client = SiftClient(
    api_key=api_key,
    grpc_url=grpc_url,
    rest_url=rest_url
)

print("✓ Sift client initialized successfully")

✓ Sift client initialized successfully

## Sift Resources

Sift objects, such as Assets, Runs, etc. are all accessed via their API resources.

The [SiftClient](../../reference/sift_client/#sift_client.SiftClient) class provides these resources as properties:
- `assets`
- `runs`
- etc.

Asynchronous versions are also available by accessing the `async_` property of the client. For example:
- `client.async_.assets`
- `client.async_.runs`
- etc.

For example, the `Ping` resource can be used for a basic health check:

In [3]:
client.ping.ping()

'Hello from Sift!'

## Assets and Runs

Assets represent physical or logical entities in your system (e.g., vehicles, machines, devices). Runs represent time-bounded operational periods for an asset (e.g., a flight, a test, a mission).

Resources generally offer similar interaction patterns and methods. For example, the `AssetsAPI` has:
- `get`
- `list_`
- `find`
- `update`
- `archive`
- `unarchive`

Other resources may offer additional methods such as `create`.

### Listing, Finding, and Getting

`list_` can be used to retrieve objects that match a specific set of criteria:

In [16]:
# List all assets (limited to 10 for this example)
assets = client.assets.list_(name_contains="Mars", limit=5)
for asset in assets:
    print(f"Name: {asset.name}, ID: {asset.id_}")


Name: MarsRoverIngestPusher, ID: 429b864b-0911-4e23-b9d1-2a1fba4d441c

Name: Mars Rover [Jonno export test], ID: 5d86ed46-dec6-41c8-8680-a0ba02d9e546

Name: MarsRover_pb4, ID: 9bb3bfec-840d-40b7-a5ab-25591f9a1b38

Name: MarsRover, ID: 611914d3-ffb1-402e-ae1e-5eb3e66dea7c

Name: MarsRover42NaN, ID: 0ad88099-1aea-461c-91b5-0a91c7811f74

`find` can be used to find a single matching object. It will return an error if multiple are found. It takes the same arguments and filters as `list_`.

In [17]:
# Find a specific asset by name
asset_name = "MarsRover0"
asset = client.assets.find(name=asset_name)
print(asset)

Asset(
    id_='61d6e4f0-8287-4678-b071-18a95fcd9db6',
    name='MarsRover0',
    organization_id='dd9f82ef-7805-4b02-9572-ec61b71edde6',
    created_date=datetime.datetime(2025, 3, 4, 19, 51, 15, 89746, tzinfo=datetime.timezone.utc),
    created_by_user_id='f47e4854-234b-421c-badb-7f8bb757cd9d',
    modified_date=datetime.datetime(2025, 3, 4, 19, 51, 15, 89746, tzinfo=datetime.timezone.utc),
    modified_by_user_id='f47e4854-234b-421c-badb-7f8bb757cd9d',
    tags=['simulator', 'speed-test'],
    metadata={'vehicle_type': 'rover', 'vehicle_version': 123123123.0, 'version_active': True},
    is_archived=False,
    archived_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc)
)

When we know exactly what we are looking for, we can use `get`.

In [18]:
# Get the exact asset by ID
asset = client.assets.get(asset_id=asset.id_)
print(asset)

Asset(
    id_='61d6e4f0-8287-4678-b071-18a95fcd9db6',
    name='MarsRover0',
    organization_id='dd9f82ef-7805-4b02-9572-ec61b71edde6',
    created_date=datetime.datetime(2025, 3, 4, 19, 51, 15, 89746, tzinfo=datetime.timezone.utc),
    created_by_user_id='f47e4854-234b-421c-badb-7f8bb757cd9d',
    modified_date=datetime.datetime(2025, 3, 4, 19, 51, 15, 89746, tzinfo=datetime.timezone.utc),
    modified_by_user_id='f47e4854-234b-421c-badb-7f8bb757cd9d',
    tags=['simulator', 'speed-test'],
    metadata={'vehicle_type': 'rover', 'vehicle_version': 123123123.0, 'version_active': True},
    is_archived=False,
    archived_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc)
)

### Creating, Updating, and Archiving

Most resources offer `create`, `update`, and `archive` methods. 

In [20]:
# List runs for the selected asset
runs = client.runs.list_(
    assets=[asset.id_],
    limit=10,
    order_by="start_time desc"
)

print(f"Found {len(runs)} runs for asset '{asset.name}':")
for run in runs:
    status = "Running" if run.stop_time is None else "Stopped"
    print(f"  - {run.name} ({status})")
    print(f"    Start: {run.start_time}")
    if run.stop_time:
        print(f"    Stop: {run.stop_time}")

AioRpcError: <AioRpcError of RPC that terminated with:
	status = StatusCode.INVALID_ARGUMENT
	details = "invalid argument: failed to compile filter: ERROR: <input>:1:25: undeclared reference to 'asset_ids' (in container '')
 | is_archived == false && asset_ids in ['61d6e4f0-8287-4678-b071-18a95fcd9db6']
 | ........................^ (a4aa7f59-a7f8-4026-b602-eeb00912c845)"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-10-10T11:51:57.213813-07:00", grpc_status:3, grpc_message:"invalid argument: failed to compile filter: ERROR: <input>:1:25: undeclared reference to \'asset_ids\' (in container \'\')\n | is_archived == false && asset_ids in [\'61d6e4f0-8287-4678-b071-18a95fcd9db6\']\n | ........................^ (a4aa7f59-a7f8-4026-b602-eeb00912c845)"}"
>

In [ ]:
# Get a specific run
if runs:
    run = runs[0]
    print(f"✓ Selected run: {run.name}")
    print(f"  ID: {run.id_}")
    print(f"  Start time: {run.start_time}")
    print(f"  Stop time: {run.stop_time or 'Still running'}")
else:
    print("No runs found for this asset")
    run = None

In [ ]:
# Filter runs by time range
recent_runs = client.runs.list_(
    assets=[asset.id_],
    start_time_after=datetime.now() - timedelta(days=7),
    limit=5
)

print(f"Runs started in the last 7 days: {len(recent_runs)}")

## Searching Channels

Channels represent time-series data streams (e.g., sensor readings, telemetry).

In [ ]:
# List channels for the selected asset
channels = client.channels.list_(
    asset=asset.id_,
    limit=20
)

print(f"Found {len(channels)} channels for asset '{asset.name}':")
for channel in channels[:10]:  # Show first 10
    print(f"  - {channel.name}")
    if channel.description:
        print(f"    Description: {channel.description}")
    if channel.units:
        print(f"    Units: {channel.units}")

In [ ]:
# Search for specific channels by name pattern
# Replace with a pattern that matches your channel names
velocity_channels = client.channels.list_(
    asset=asset.id_,
    name_contains="velocity",
    limit=10
)

print(f"Channels containing 'velocity': {len(velocity_channels)}")
for ch in velocity_channels:
    print(f"  - {ch.name}")

In [ ]:
# Get channels for a specific run
if run:
    run_channels = client.channels.list_(
        run=run.id_,
        limit=10
    )
    print(f"Channels in run '{run.name}': {len(run_channels)}")
    for ch in run_channels:
        print(f"  - {ch.name}")

## Pulling Data

Retrieve time-series data from channels as pandas DataFrames.

In [ ]:
# Get data for specific channels
if channels and run:
    # Select first 3 channels for this example
    selected_channels = channels[:3]

    print(f"Fetching data for {len(selected_channels)} channels...")

    # Get data as a dictionary of pandas DataFrames
    data = client.channels.get_data(
        channels=selected_channels,
        run=run.id_,
        limit=1000  # Limit to 1000 data points per channel
    )

    print(f"\n✓ Retrieved data for {len(data)} channels:")
    for channel_name, df in data.items():
        print(f"\n  Channel: {channel_name}")
        print(f"  Data points: {len(df)}")
        if len(df) > 0:
            print(f"  Columns: {list(df.columns)}")
            print(f"  Sample data:")
            print(df.head())
else:
    print("No channels or run available to fetch data")

In [ ]:
# Get data for a specific time range
if channels and run and run.start_time:
    selected_channels = channels[:2]

    # Get data for first hour of the run
    start_time = run.start_time
    end_time = start_time + timedelta(hours=1)

    print(f"Fetching data from {start_time} to {end_time}...")

    data = client.channels.get_data(
        channels=selected_channels,
        run=run.id_,
        start_time=start_time,
        end_time=end_time
    )

    print(f"\n✓ Retrieved time-ranged data:")
    for channel_name, df in data.items():
        print(f"  {channel_name}: {len(df)} data points")

## Creating Calculated Channels

Calculated channels allow you to create derived metrics from existing channels using mathematical expressions.

In [ ]:
# Create a calculated channel
# This example creates a channel that divides two existing channels
# Replace channel names with actual channels from your system

if len(channels) >= 2:
    # Use first two channels for this example
    channel1 = channels[0]
    channel2 = channels[1]

    calc_channel_name = f"{channel1.name}_per_{channel2.name}"

    # Check if calculated channel already exists
    existing = client.calculated_channels.find(
        name=calc_channel_name,
        asset=asset.id_
    )

    if existing:
        print(f"Calculated channel '{calc_channel_name}' already exists")
        calc_channel = existing
    else:
        print(f"Creating calculated channel: {calc_channel_name}")

        calc_channel = client.calculated_channels.create(
            CalculatedChannelCreate(
                name=calc_channel_name,
                description=f"Ratio of {channel1.name} to {channel2.name}",
                expression="$1 / $2",  # $1 and $2 refer to the channel references below
                channel_references=[
                    ChannelReference(
                        channel_reference="$1",
                        channel_identifier=channel1.name
                    ),
                    ChannelReference(
                        channel_reference="$2",
                        channel_identifier=channel2.name
                    ),
                ],
                units=f"{channel1.units or 'unit1'}/{channel2.units or 'unit2'}",
                asset_ids=[asset.id_],
            )
        )

        print(f"✓ Created calculated channel: {calc_channel.name}")
        print(f"  ID: {calc_channel.id_}")
        print(f"  Expression: {calc_channel.expression}")
else:
    print("Not enough channels available to create a calculated channel")
    calc_channel = None

In [ ]:
# List all calculated channels for the asset
calc_channels = client.calculated_channels.list_(
    asset=asset.id_,
    limit=10
)

print(f"Calculated channels for asset '{asset.name}': {len(calc_channels)}")
for cc in calc_channels:
    print(f"  - {cc.name}")
    print(f"    Expression: {cc.expression}")
    print(f"    Version: {cc.version}")

## Setting Rules

Rules allow you to define conditions that trigger actions (like creating annotations) when met.

In [ ]:
# Create a rule that monitors a channel or calculated channel
# This example creates a rule that triggers when a value exceeds a threshold

if calc_channel:
    rule_name = f"high_{calc_channel.name}_alert"

    # Check if rule already exists
    existing_rule = client.rules.find(name=rule_name)

    if existing_rule:
        print(f"Rule '{rule_name}' already exists")
        rule = existing_rule
    else:
        print(f"Creating rule: {rule_name}")

        rule = client.rules.create(
            RuleCreate(
                name=rule_name,
                description=f"Alert when {calc_channel.name} exceeds threshold",
                expression="$1 > 10",  # Adjust threshold as needed
                channel_references=[
                    ChannelReference(
                        channel_reference="$1",
                        channel_identifier=calc_channel.name
                    ),
                ],
                action=RuleAction.annotation(
                    annotation_type=RuleAnnotationType.DATA_REVIEW,
                    tags=["high_value", "alert"],
                    default_assignee_user_id=None,
                ),
                asset_ids=[asset.id_],
            )
        )

        print(f"✓ Created rule: {rule.name}")
        print(f"  ID: {rule.id_}")
        print(f"  Expression: {rule.expression}")
        print(f"  Enabled: {rule.is_enabled}")
elif channels:
    # Create a rule using a regular channel
    channel = channels[0]
    rule_name = f"high_{channel.name}_alert"

    existing_rule = client.rules.find(name=rule_name)

    if existing_rule:
        print(f"Rule '{rule_name}' already exists")
        rule = existing_rule
    else:
        print(f"Creating rule: {rule_name}")

        rule = client.rules.create(
            RuleCreate(
                name=rule_name,
                description=f"Alert when {channel.name} exceeds threshold",
                expression="$1 > 100",  # Adjust threshold as needed
                channel_references=[
                    ChannelReference(
                        channel_reference="$1",
                        channel_identifier=channel.name
                    ),
                ],
                action=RuleAction.annotation(
                    annotation_type=RuleAnnotationType.DATA_REVIEW,
                    tags=["threshold_exceeded"],
                    default_assignee_user_id=None,
                ),
                asset_ids=[asset.id_],
            )
        )

        print(f"✓ Created rule: {rule.name}")
else:
    print("No channels available to create a rule")
    rule = None

In [ ]:
# List all rules for the asset
rules = client.rules.list_(
    asset_ids=[asset.id_],
    limit=10
)

print(f"Rules for asset '{asset.name}': {len(rules)}")
for r in rules:
    status = "Enabled" if r.is_enabled else "Disabled"
    print(f"  - {r.name} ({status})")
    print(f"    Expression: {r.expression}")
    if r.action:
        print(f"    Action: {r.action.action_type.name}")

## Summary

This notebook demonstrated:
1. ✓ Initializing the Sift client with API credentials
2. ✓ Finding and filtering assets
3. ✓ Finding and filtering runs
4. ✓ Searching for channels by various criteria
5. ✓ Pulling time-series data as pandas DataFrames
6. ✓ Creating calculated channels with mathematical expressions
7. ✓ Setting up rules with conditions and actions

### Next Steps
- Explore more filtering options for assets, runs, and channels
- Create more complex calculated channels with advanced expressions
- Set up rules with different action types (webhooks, etc.)
- Visualize the data using matplotlib or plotly
- Use the async API for better performance in production applications

In [ ]:
# Optional: Clean up resources
# Uncomment to archive the created calculated channel and rule

# if calc_channel:
#     calc_channel.archive()
#     print(f"Archived calculated channel: {calc_channel.name}")

# if rule:
#     rule.archive()
#     print(f"Archived rule: {rule.name}")

print("\n✓ Example complete!")